<img src="https://github.com/denisabrantes/pdk-use-cases/blob/main/images/hpe_logo.png?raw=true" alt="HPE Logo" width="250" style="align:left;margin-left:0;"/>

<h1>Object Detection Prediction from KServe InferenceService</h1>

<b>Date: 01/02/24</b><br/>
<b>Version: 0.1</b><br/>
<b>Authors: HPE AI At Scale SE Team</b>

<img src="https://github.com/denisabrantes/pdk-use-cases/blob/main/images/platform_step0.png?raw=true" alt="Enterprise Machine Learning platform architecture" width="850">

<h3>Import modules and define functions</h3>
The cells below imports all modules and libraries required to run the demo.

In [ ]:
# !pip install -q ipywidgets
# !pip install -q scikit-image

In [1]:
import sys
import glob
import base64
import json
import requests
import matplotlib.pyplot as plt

from skimage import io
from PIL import Image, ImageDraw
from ipywidgets import interact, interactive
import ipywidgets as widgets
import io

from tqdm import tqdm
from multiprocessing import Pool

<h2>Part 1: Use this code to load images from a folder and generate predictions</h2>

<h4>Step 1: Setting up connection details to KServe</h4>

In [2]:
endpoint_name = "detection-deploy"
model_name ="object-detection"
ingress_host = "192.168.1.2"
ingress_port = "80"
service_hostname = "detection-deploy.models.example.com"

<h4>Step 2: Request prediction from KServe InferenceService</h4>

In [3]:
def plot_pred(im,pred_d,thres=0.15):
    draw = ImageDraw.Draw(im)
    try:
        for pred in pred_d['predictions'][0]:
            assert len(list(pred.keys())) == 2
            cl_name = list(pred.keys())[0]
            bboxes = pred[cl_name]
            if pred['score'] > thres:
                draw.rectangle([bboxes[0],bboxes[1],bboxes[2],bboxes[3]],outline=(255,0,0),fill=None,width=1)
                draw.text([bboxes[0],bboxes[1]-10],"{} :{:.2f}".format(cl_name,pred['score']),fill=(250,0,0))
        plt.figure(figsize=(8,8))
    except Exception as e:
        print(e)
        pass
    plt.imshow(im)
    plt.show()
    return im

In [4]:
def predict(args):
    image= Image.open(args[0])
    with io.BytesIO() as buffer:
        image.save(buffer, format='JPEG')
        image_bytes = buffer.getvalue()
    image_64_encode = base64.b64encode(image_bytes)
    bytes_array = image_64_encode.decode("utf-8")
    
    # Format the request in json
    request = {
      "instances":[
        {
          "data": bytes_array
        }
      ]
    }
    ingress_host = args[1]
    ingress_port = args[2]
    model_name = args[3]
    service_hostname = args[4]

    url = str("http://") + str(ingress_host) + ":" + str(ingress_port) + "/v1/models/" + str(model_name) + ":predict"
    headers = {'Host': service_hostname, "Content-Type" : "application/json"}
    payload = json.dumps(request)

    response = requests.post(url, data=payload, headers=headers)
    return response.json()

In [5]:
def visualize(idx,thres=0.15):
    print(idx,thres)
    output = resps[idx]
    im = Image.open(imgs[idx])
    plot_pred(im,output,thres)

In [6]:
def run_apply_async_multiprocessing(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)

    jobs = [pool.apply_async(func=func, args=(*argument,)) if isinstance(argument, tuple) else pool.apply_async(func=func, args=(argument,)) for argument in argument_list]
    pool.close()
    result_list_tqdm = []
    for job in tqdm(jobs):
        result_list_tqdm.append(job.get())

    return result_list_tqdm

In [7]:
images = r"./images/*.jpg"

In [8]:
imgs = [img for img in glob.glob(images, recursive=True)]

In [9]:
resps = run_apply_async_multiprocessing(predict,
                                        [[imgs[i],
                                        ingress_host,
                                        ingress_port,
                                        model_name,
                                        service_hostname] for i in range(len(imgs))],num_processes=4)

100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


In [10]:
interact(visualize, 
         idx=widgets.IntSlider(min=0, max=len(resps), step=1, value=0),
         thres = widgets.FloatSlider(min=0, max=1.0, step=0.1, value=0.00));

interactive(children=(IntSlider(value=0, description='idx', max=1), FloatSlider(value=0.0, description='thres'…

<h2>Part 2: Use this code to load the sample .json files and generate predictions</h2>

In [11]:
with open("object_detection.json", "r") as objfile:
    imgfile = json.loads(objfile.read())

In [12]:
jsonimg = imgfile["instances"][0]["data"]

In [13]:
# Format the request in json
request = {
  "instances":[
    {
      "data": jsonimg
    }
  ]
}

url = str("http://") + str(ingress_host) + ":" + str(ingress_port) + "/v1/models/" + str(model_name) + ":predict"
headers = {'Host': service_hostname, "Content-Type" : "application/json"}
payload = json.dumps(request)

response = requests.post(url, data=payload, headers=headers)

In [14]:
print(response.json())

{'predictions': [[{'Cargo Plane': [620.5451049804688, 548.1950073242188, 634.9935302734375, 559.5702514648438], 'score': 0.5663272142410278}, {'Cargo Plane': [0.888470470905304, 163.86721801757812, 120.9725112915039, 233.10594177246094], 'score': 0.5661454796791077}, {'Cargo Plane': [616.62646484375, 546.4703979492188, 633.0997314453125, 556.7673950195312], 'score': 0.5650514960289001}, {'Cargo Plane': [615.9215698242188, 551.2975463867188, 633.3138427734375, 563.1387329101562], 'score': 0.5572015047073364}, {'Cargo Plane': [0.0, 107.6839370727539, 194.9051971435547, 256.3356628417969], 'score': 0.5497938394546509}, {'Cargo Plane': [623.7868041992188, 550.5260620117188, 637.169189453125, 565.0270385742188], 'score': 0.5360344052314758}, {'Cargo Plane': [611.221923828125, 562.0796508789062, 631.2430419921875, 572.4779052734375], 'score': 0.5338000655174255}, {'Cargo Plane': [630.8724365234375, 455.24151611328125, 638.4213256835938, 477.82745361328125], 'score': 0.5337476134300232}, {'Ca